In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from matplotlib import pyplot
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk import word_tokenize
import nltk
from data_loading_code import preprocess_pandas
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, classification_report


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device is", device)

device is cpu


In [82]:
#nltk.download('punkt')
#nltk.download('stopwords')
# get data, pre-process and split
data = pd.read_csv("amazon_cells_labelled.txt", delimiter='\t', header=None)
data.columns = ['Sentence', 'Class']
data['index'] = data.index                                          # add new column index
columns = ['index', 'Class', 'Sentence']
data = preprocess_pandas(data, columns)                             # pre-process
training_data, validation_data, training_labels, validation_labels = train_test_split( # split the data into training, validation, and test splits
    data['Sentence'].values.astype('U'),
    data['Class'].values.astype('int32'),
    test_size=0.10,
    random_state=0,
    shuffle=True
)

print(training_data)
print(type(training_data[0]))

# vectorize data using TFIDF and transform for PyTorch for scalability
word_vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,2), max_features=50000, max_df=0.5, use_idf=True, norm='l2')
training_data = word_vectorizer.fit_transform(training_data)        # transform texts to sparse matrix
training_data = training_data.todense()                             # convert to dense matrix for Pytorch
vocab_size = len(word_vectorizer.vocabulary_)
validation_data = word_vectorizer.transform(validation_data)
validation_data = validation_data.todense()

train_x_tensor = torch.from_numpy(np.array(training_data)).type(torch.FloatTensor)
train_y_tensor = torch.from_numpy(np.array(training_labels)).long()
validation_x_tensor = torch.from_numpy(np.array(validation_data)).type(torch.FloatTensor)
validation_y_tensor = torch.from_numpy(np.array(validation_labels)).long()

<U149
<class 'numpy.str_'>


In [31]:
print(train_x_tensor.shape)

torch.Size([900, 7277])


In [44]:
train_x_tensor[1].shape

torch.Size([7277])

In [32]:
print(data.iloc(0))

In [33]:
dl = DataLoader(train_y_tensor, batch_size=5,num_workers=2)
it = iter(dl)
print(next(it))

tensor([0, 1, 0, 1, 1])


In [34]:
class ConcatDataset(torch.utils.data.Dataset):
    def __init__(self, datasetA, datasetB):
        self.datasetA = datasetA
        self.datasetB = datasetB

    def __getitem__(self, i):
        inp = self.datasetA[i]
        label = F.one_hot(self.datasetB[i],num_classes=2)
        return inp,label

    def __len__(self):
        return min(len(self.datasetA),len(self.datasetB))

In [35]:
train_ds = ConcatDataset(train_x_tensor,train_y_tensor)
val_ds = ConcatDataset(validation_x_tensor,validation_y_tensor)
train_loader = DataLoader(train_ds,batch_size=5)
val_loader = DataLoader(val_ds,batch_size=5)

In [36]:
class ANN(nn.Module):
    def __init__(self,num_classes):
        super().__init__()
        
        self.act = nn.LeakyReLU()

        # input: 7277        
        self.fc1 = nn.Linear(in_features=7277, out_features=1000) 
        self.fc2 = nn.Linear(in_features=1000, out_features=100)
        self.fc3 = nn.Linear(in_features=100, out_features=10)
        self.fc4 = nn.Linear(in_features=10, out_features=2)
        self.Softmax = nn.Softmax(dim=1)


    
    def forward(self, x):
        #FC Layer 1
        x = self.fc1(x)
        x = self.act(x)

        #FC Layer 2
        x = self.fc2(x)   
        x = self.act(x)    

        #FC Layer 3
        x = self.fc3(x)   
        x = self.act(x)    

        #FC Layer 4
        x = self.fc4(x)    
        
        #Softmax
        out = self.Softmax(x)

        return out

In [37]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs):
    best_val_loss = float('inf')
    best_model = model

    for epoch in range(num_epochs):
        print(f"Starting epoch {epoch+1} of {num_epochs}")
        model.train()
        train_loss = 0.0
        for batch_nr, (inputs, labels) in enumerate(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device).to(torch.float)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss = loss.to(device)

            if (batch_nr%20 == 0):
                print(f"Processing batch number {batch_nr+1} of {len(train_loader)}")
                print("current loss",loss.item())
                
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
        
        train_loss /= len(train_loader.dataset)
    
    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device).to(torch.float)
            outputs = model(inputs)
        
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
    
    val_loss /= len(val_loader.dataset)
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    
    # Save the best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model
    return best_model

In [38]:
LEARNING_RATE = 0.0001
EPOCHS = 3

print(device)
model = ANN(num_classes=2).to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
# optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Train the model
trained_model = train_model(model, criterion, optimizer, train_loader, val_loader, EPOCHS)

cpu
Starting epoch 1 of 3
Processing batch number 1 of 180
current loss 0.695035994052887
Processing batch number 21 of 180
current loss 0.6946902275085449
Processing batch number 41 of 180
current loss 0.694510281085968
Processing batch number 61 of 180
current loss 0.6885390281677246
Processing batch number 81 of 180
current loss 0.694531261920929
Processing batch number 101 of 180
current loss 0.6974092125892639
Processing batch number 121 of 180
current loss 0.6933392286300659
Processing batch number 141 of 180
current loss 0.693509042263031
Processing batch number 161 of 180
current loss 0.690544605255127
Starting epoch 2 of 3
Processing batch number 1 of 180
current loss 0.6902028322219849
Processing batch number 21 of 180
current loss 0.6882875561714172
Processing batch number 41 of 180
current loss 0.6797265410423279
Processing batch number 61 of 180
current loss 0.6751033067703247
Processing batch number 81 of 180
current loss 0.6606310606002808
Processing batch number 101 of 

In [114]:
while True:
    inp = input("Leave a review:")
    if inp=="exit":
        break
    tdata = word_vectorizer.transform([" ".join(word_tokenize(inp))])
    tdata = torch.Tensor(tdata.todense())
    tdata = torch.from_numpy(np.array(tdata)).type(torch.FloatTensor)
    print(tdata.shape)
    pred = model(tdata).tolist()
    if(pred[0][0]>0.5):
        print("You seem to dislike this thing")
    else:
        print("You seem to like this thing")

torch.Size([1, 7277])
You seem to dislike this thing
torch.Size([1, 7277])
You seem to like this thing
